In [82]:
!pip install torch

import torch.nn as nn
import pandas as pd

In [70]:
df = pd.read_csv("bcancer_data.csv")

In [71]:
df.head()

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
0,1,1,1,1,2,1,2,1,1,0
1,5,1,1,1,1,1,3,1,1,0
2,4,1,1,1,2,1,1,1,1,0
3,2,1,1,1,2,1,2,1,1,0
4,1,2,3,1,2,1,2,1,1,0


In [72]:
df.shape

(540, 10)

In [73]:
df.Class.value_counts()

Class
0    301
1    239
Name: count, dtype: int64

In [74]:
df = df.to_numpy()
df

array([[1, 1, 1, ..., 1, 1, 0],
       [5, 1, 1, ..., 1, 1, 0],
       [4, 1, 1, ..., 1, 1, 0],
       ...,
       [4, 1, 1, ..., 1, 1, 0],
       [2, 1, 1, ..., 1, 1, 0],
       [4, 8, 8, ..., 4, 1, 1]])

In [100]:
X = df[:, :5]   
y = df[:, 9]    


In [101]:
X.shape, y.shape

((540, 5), (540,))

In [102]:
X_sample = X[0]
y_sample = y[0]

print(X_sample, y_sample)


[1 1 1 1 2] 0


In [103]:
import torch
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X[:5], y[:5]

(tensor([[1., 1., 1., 1., 2.],
         [5., 1., 1., 1., 1.],
         [4., 1., 1., 1., 2.],
         [2., 1., 1., 1., 2.],
         [1., 2., 3., 1., 2.]]),
 tensor([0., 0., 0., 0., 0.]))

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


len(X_train), len(y_train), len(X_test), len(y_test)

(432, 432, 108, 108)

In [108]:
class bcancerclassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(5,10)
        self.act1 = nn.ReLU()
        self.output = nn.Linear(10,1)
        self.act_output = nn.Sigmoid()
    
    def forward(self,x):
        x = self.act1(self.hidden1(x))
        x = self.act_output(self.output(x))
        return x        


In [109]:
model = bcancerclassifier()
print(model)

bcancerclassifier(
  (hidden1): Linear(in_features=5, out_features=10, bias=True)
  (act1): ReLU()
  (output): Linear(in_features=10, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [110]:
untrained_preds = model(X_test)
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(y_test)}, Shape: {y_test.shape}")
print(f"\nFirst 10 predictions:\n{untrained_preds[:10]}")
print(f"\nFirst 10 test labels:\n{y_test[:10]}")

Length of predictions: 108, Shape: torch.Size([108, 1])
Length of test samples: 108, Shape: torch.Size([108])

First 10 predictions:
tensor([[0.2824],
        [0.5091],
        [0.5066],
        [0.6078],
        [0.3418],
        [0.5066],
        [0.3958],
        [0.4858],
        [0.5163],
        [0.4808]], grad_fn=<SliceBackward0>)

First 10 test labels:
tensor([1., 0., 0., 0., 1., 0., 1., 0., 1., 0.])


In [122]:
loss_fn = nn.BCEWithLogitsLoss
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.1)

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [119]:
y_logits = model(X_test)[:5]
y_logits

tensor([[0.2824],
        [0.5091],
        [0.5066],
        [0.6078],
        [0.3418]], grad_fn=<SliceBackward0>)